<a href="https://www.hydroffice.org/epom/"><img src="images/000_000_epom_logo.png" alt="ePOM" title="Open ePOM home page" align="center" width="12%" alt="Python logo\"></a>

<a href="https://piazza.com/e-learning_python_for_ocean_mapping/fall2019/om100/home"><img src="images/help.png" alt="ePOM" title="Ask questions on Piazza.com" align="right" width="10%" alt="Piazza.com\"></a>
# Introduction to GDAL

<img align="left" width="6%" style="padding-right:10px;" src="images/key.png">

The [Geospatial Data Abstraction Lybrary (GDAL)](https://gdal.org) is a key component of many geospatial processing workflows.

GDAL provides a simplified means to access an increasing number of geospatial [raster](https://en.wikipedia.org/wiki/GIS_file_formats#Raster) formats (see the [GDAL format list](https://gdal.org/formats_list.html)) and to modify the retrieved data (e.g., changing their cartographic projections). The library also supports several [vector](https://en.wikipedia.org/wiki/GIS_file_formats#Vector) formats through the `ogr` component (see [OGR vector formats](https://gdal.org/ogr_formats.html)). 

The library is written in [C++](https://en.wikipedia.org/wiki/C%2B%2B), but can be used in Python and in [many other popular computer languages](https://trac.osgeo.org/gdal/#GDALOGRInOtherLanguages).

Before starting to use `gdal`, you have to execute the following cell that, together with code introduced in the past notebooks, imports `gdal` from the `osgeo` namespace:

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from osgeo import gdal
from osgeo import ogr
from osgeo import osr

sys.path.append(os.getcwd())
np.set_printoptions(precision=2, floatmode='fixed')

The cell below retrieves and prints the GDAL version:

In [ ]:
print("GDAL version: %s" % (gdal.__version__, ))

---

# The BAG Format

<img align="left" width="6%" style="padding-right:10px;" src="images/key.png">

The [Bathymetric Attributed Grid](https://en.wikipedia.org/wiki/Bathymetric_attributed_grid) (BAG) is a raster file format designed to store and exchange bathymetric data. The BAG format aims to ensure interoperability amongst vendors and agencies. 

The BAG format was developed and is currently maintained by [The Open Navigation Surface](http://www.opennavsurf.org/) working group, with members from [CARIS](http://www.teledynecaris.com), [CCOM/JHC](http://www.ccom.unh.edu/), [ESRI](https://www.esri.com), [NAVO](https://www.usno.navy.mil/NAVO), [NOAA](https://www.noaa.gov/), [QPS](https://qps.nl), [SAIC](http://www.saic.com/) and several other organizations.

The BAG data structure is based on the flexibility provided by the [Hierarchical Data Format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) (HDF) format.

<img align="left" width="6%" style="padding-right:10px;" src="images/key.png">

There are four mandatory components in a BAG file:
<br><br>

* Elevation layer
* Uncertainty layer
* Tracking List table (with all the modifications applied to the original data)
* Metadata string (e.g., projection, resolution, data lineage) in the ISO Metadata [XML](https://en.wikipedia.org/wiki/XML) format 

![BAG Format](images/IO_000_000_bag_format.png)

You can use the [BAG Explorer](https://www.hydroffice.org/bag/main) application to explore the content of a BAG file:

![BAG Explorer](images/IO_000_001_bag_explorer.png)

<img align="left" width="6%" style="padding-right:10px;" src="images/key.png">

GDAL supports the BAG raster format with the limitations described on [this page](https://www.gdal.org/frmt_bag.html).

Following some of the steps described in the official [GDAL tutorial](https://www.gdal.org/gdal_tutorial.html), the cell below explores the content of a BAG file.

In [ ]:
bag_path = os.path.join(os.getcwd(), "data", "sh2007.bag")  # the 'sh2007.bag' file is located under the `data` folder

dataset = gdal.Open(bag_path, gdal.GA_ReadOnly)
if not dataset:
    raise RuntimeError("Issue in opening the BAG file: %s" % bag_path)

print("BAG file was successfully opened: %s" % (bag_path,))

## Dataset Information

A GDAL **dataset** contains:

* Several **raster bands**, co-located and at the same resolution.
* **Metadata** information.
* A **projection**, that is the coordinate reference system in use by the stored data.
* A **geo-transform**, required to convert the grid indices into geographical coordinates.
* Other information (e.g., layer shape).

The cell below retrieves and displays some of the available dataset information:

In [ ]:
print("GDAL driver name: %s" % dataset.GetDriver().LongName)

print("Number of bands: %s" % dataset.RasterCount)
print("Band shape: %s x %s" % (dataset.RasterXSize, dataset.RasterYSize,))

Based on the results from the cell above, GDAL identified two layers in the BAG file. This is in agreement with the [GDAL documentation for the BAG format](https://www.gdal.org/frmt_bag.html): *"BAG files have two or three image bands representing Elevation (band 1), Uncertainty (band 2) and Nominal Elevation (band 3) values for each cell in a raster grid area."*.  Thus, in this specific case, we can retrieve the first two bands: the elevation and the uncertainty. 

Other useful information is the shape of the arrays in those bands: `841 x 1246`.

The cells below retrieve and show the BAG projection in [Well Known Text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_coordinate_reference_systems) as well as its official name (e.g., `WGS 84 / UTM zone 19N`):

In [ ]:
print("Projection:")
projection = dataset.GetProjection()
print("- WKT:\n%s" % projection)

srs = osr.SpatialReference(wkt=projection)
projection_name = srs.GetAttrValue('projcs')
print("- name: %s" % projection_name)

The cells below retrieve and show the six elements of a geo-transform (for more details, read the [GDAL Data Model page](https://www.gdal.org/gdal_datamodel.html)):

* `gt[0]`: x coordinate for the top-left corner
* `gt[1]`: grid resolution in the west-east direction.
* `gt[2]`: 0 for north-up raster
* `gt[3]`: y coordinate for the top-left corner
* `gt[4]`: 0 for north-up raster
* `gt[5]`: grid resolution in the north-south direction (usually, a negative value)

In [ ]:
gt = dataset.GetGeoTransform()
if not gt:
    raise RuntimeError("Issue in retrieving the geotransfrom")
print("Geo-Transform: [%.2f, %.2f, %.2f, %.2f, %.2f, %.2f]" % gt)

print("Top-left Corner = (%.2f, %.2f)" % (gt[0], gt[3]))
print("Grid Resolution = (%.2f, %.2f)" % (gt[1], gt[5]))

The cell below retrieves, from the BAG file, the string containing the XML metadata:

In [ ]:
metadata = dataset.GetMetadata("xml:BAG")[0]
print("metadata:\n%s" % metadata)

<img align="left" width="6%" style="padding-right:10px;" src="images/info.png">

To make the text more readable, you can use XML-aware packages such as [`xml.dom.minidom`](https://docs.python.org/3.6/library/xml.dom.minidom.html):

In [ ]:
from xml.dom import minidom

dom = minidom.parseString(metadata)
print(dom.toprettyxml())

<img align="left" width="6%" style="padding-right:10px;" src="images/info.png">

Once that the metadata are parsed using [`xml.dom.minidom`](https://docs.python.org/3.6/library/xml.dom.minidom.html), you can retrieve specific metadata components:

In [ ]:
org_name = dom.getElementsByTagName("gmd:organisationName")[0]
print("The XML element with the organization that has created the BAG file:\n%s" % org_name.toprettyxml())

## Reading Raster Layers as Numpy Arrays

The cell below retrieves the two raster bands from the dataset (the numbering starts from 1), and then displays some information about them.

In [ ]:
elev_band = dataset.GetRasterBand(1)
print("Elevation band -> data type: %s" % (gdal.GetDataTypeName(elev_band.DataType)))
print("Elevation band -> min/max values in meters: %.3f/%.3f" % (elev_band.GetMinimum(), elev_band.GetMaximum()))
elev_nodata = elev_band.GetNoDataValue()
print("Elevation band -> nodata value: %s" % (elev_nodata))

unc_band = dataset.GetRasterBand(2)
print("Uncertainty band -> data type: %s" % (gdal.GetDataTypeName(unc_band.DataType)))
print("Uncertainty band -> min/max values in meters: %.3f/%.3f" % (unc_band.GetMinimum(), unc_band.GetMaximum()))
unc_nodata = elev_band.GetNoDataValue()
print("Uncertainty band -> nodata value: %s" % (unc_nodata))

For each of the two bands, the cell above provides:

* The data type: `Float32`.
* The range of values (i.e., the minimum and the maximum values).
* The value used to mark a grid cell without data: 1,000,000.

The cell below will read the elevation band as a NumPy array:

In [ ]:
elev = elev_band.ReadAsArray()
print("maximum elevation: %.3f" % np.nanmax(elev))
print("minimum elevation: %.3f" % np.nanmin(elev))

plt.imshow(elev)
plt.show()

The plot above has only two colors. *It does not look like valid bathymetry!*

The minimum value should give a good hint of what is going on. The no-data values have been plotted! The cell below will put `np.nan` in place of those values. Furthermore, we convert the elevation values into bathymetry by inverting their sign. 

In [ ]:
elev = elev_band.ReadAsArray()
elev[elev==elev_nodata] = np.nan
elev = -elev

print("minimum depth: %.3f" % np.nanmin(elev))
print("maximum depth: %.3f" % np.nanmax(elev))

plt.imshow(elev)
plt.show()

The plot above looks much better! 

The cell below calculates the extent of the data based on the geo-transform values:

In [ ]:
bag_extent = [gt[0], gt[0] + elev.shape[1] * gt[1], gt[3] + elev.shape[0] * gt[5], gt[3]]
print("BAG extent: %s" % (bag_extent, ))

We can now finalize the BAG plot with the retrieved data extent and a colorbar: 

In [ ]:
plt.figure(figsize=(15, 10))  # to make the plot bigger
ax = plt.axes()
plt.ticklabel_format(useOffset=False)  # to avoid the display of an offset for the labels
ax.xaxis.set_major_locator(ticker.MultipleLocator(100.0))  # to set the distance between labels (x axis)
ax.yaxis.set_major_locator(ticker.MultipleLocator(100.0))  # to set the distance between labels (y axis)
img = ax.imshow(elev, origin='upper', extent=bag_extent, cmap='viridis_r')  # display the array using the passed data extent
ax.set_aspect('equal')  # avoid deformations for the displayed data
title = "%s - Bathymetry [meter]\nProjection: %s" % (os.path.basename(bag_path), projection_name)
plt.title(title)  # put the filename and the projection as part of the title
plt.xlabel("Easting") 
plt.ylabel("Northing")
plt.colorbar(img)  # pass the 'img' object used to create the colorbar
plt.grid()
plt.show()

The plot above shows an area with sand ripples in Portsmouth Harbor, New Hampshire (USA).

<img align="left" width="6%" style="padding-right:10px;" src="images/test.png">

Create a plot for the uncertainty layer similar to the one generated for bathymetry. What are the areas with higher uncertainty?

In [ ]:
unc = unc_band.ReadAsArray()
unc[unc==unc_nodata] = np.nan
print("minimum uncertainty: %.3f" % np.nanmin(unc))
print("maximum uncertainty: %.3f" % np.nanmax(unc))

bag_extent = [gt[0], gt[0] + unc.shape[1] * gt[1], gt[3] + unc.shape[0] * gt[5], gt[3]]
print("BAG extent: %s" % (bag_extent, ))

plt.figure(figsize=(15, 10))
ax = plt.axes()
plt.ticklabel_format(useOffset=False)
ax.xaxis.set_major_locator(ticker.MultipleLocator(100.0))
ax.yaxis.set_major_locator(ticker.MultipleLocator(100.0))
img = ax.imshow(unc, origin='upper', extent=bag_extent)
ax.set_aspect('equal') 
title = "%s - Uncertainty [meter]\nProjection: %s" % (os.path.basename(bag_path), projection_name)
plt.title(title)  # put the filename and the projection as part of the title
plt.xlabel("Easting") 
plt.ylabel("Northing")
plt.colorbar(img)
plt.grid()

In [ ]:
unc = unc_band.ReadAsArray()

 ***

Do you know that the [U.S. national archive for multibeam bathymetric data](https://www.ngdc.noaa.gov/mgg/bathymetry/multibeam.html) is publicly available? The Multibeam Bathymetry Database (MBBDB) includes tons of hydrographic multibeam survey data from NOAA's National Ocean Service (NOS).

Visit the [Bathymetric Data Viewer](https://maps.ngdc.noaa.gov/viewers/bathymetry/), download one or more BAGs from an area of your choice, and plot the retrieved data!

<a href="https://maps.ngdc.noaa.gov/viewers/bathymetry/"><img src="images/IO_000_0002_bathymetric_data_viewer.png" alt="Bathymetric Data Viewer" title="Open the Bathymetric Data Viewer page" align="center" width="80%"></a>

***

<img align="left" width="6%" style="padding-right:10px; padding-top:10px;" src="images/refs.png">

## Useful References

* [The official Python 3.6 documentation](https://docs.python.org/3.6/index.html)
* [Programming Basics with Python](https://github.com/hydroffice/python_basics)
* The GDAL Package:
  * [Website](https://gdal.org)
  * [Data Model page](https://www.gdal.org/gdal_datamodel.html)
  * [Raster Formats](https://gdal.org/formats_list.html)
  * [Vector Formats](https://gdal.org/ogr_formats.html)
  * [Tutorial](https://www.gdal.org/gdal_tutorial.html)
* [Bathymetric Attributed Grid](https://en.wikipedia.org/wiki/Bathymetric_attributed_grid)
* [The Open Navigation Surface](http://www.opennavsurf.org/)
* [Hierarchical Data Format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format)
* [WKT for Coordinate Reference Systems](https://en.wikipedia.org/wiki/Well-known_text_representation_of_coordinate_reference_systems)
* [Bathymetric Data Viewer](https://maps.ngdc.noaa.gov/viewers/bathymetry/)

<img align="left" width="5%" style="padding-right:10px;" src="images/email.png">

*For issues or suggestions related to this notebook, write to: epom@ccom.unh.edu*

<!--NAVIGATION-->
| [Contents](index.ipynb) | [Adopting NumPy in Class Methods >](COMP_001_Adopting_NumPy_in_Class_Methods.ipynb)